In [31]:
import pandas as pd
import numpy as np
import os
import pickle

In [22]:
%store -r gdf_annual_with_svi
%store -r gdf_quarter_with_svi
%store -r gdf_semi_with_svi

gdf_annual = gdf_annual_with_svi

In [23]:
data_path = '/Users/jyontika/Desktop/Python/github_hughes/opioid-overdose-models/CASTNet/hughes-CASTNet/Data/cook-county-data/'
gdf_annual.columns

Index(['geoid', 'year', 'deaths', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'NAME',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'lat', 'lon',
       'geometry', 'timestep', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc',
       'theme_4_pc', 'svi_pctile'],
      dtype='object')

In [24]:
geoid_data = []

for geoid in gdf_annual['geoid'].unique():
    geoid_df = gdf_annual[gdf_annual['geoid'] == geoid]
    
    # Extract the deaths for lookback years
    deaths_2020 = geoid_df[geoid_df['year'] == 2020]['deaths'].values[0]
    deaths_2019 = geoid_df[geoid_df['year'] == 2019]['deaths'].values[0]
    deaths_2018 = geoid_df[geoid_df['year'] == 2018]['deaths'].values[0]
    
    # Extract the lat and lon values 
    lat = geoid_df['lat'].values[0]
    lon = geoid_df['lon'].values[0]
    
    # append data
    geoid_data.append([geoid, deaths_2020, deaths_2019, deaths_2018, lat, lon])


static_features = pd.DataFrame(geoid_data)
static_features = static_features.transpose()
csv_path = os.path.join(data_path, 'static_features.csv')
static_features.to_csv(csv_path, index=False, header=False)



In [27]:
geoid_series = gdf_annual['geoid']
txt_path = os.path.join(data_path, 'locations.txt')
with open(txt_path, 'w') as file:
    for geoid in geoid_series:
        file.write(str(geoid) + '\n')


### Add SVI as a pickle file

In [29]:
##SVI  dynamic features organization
svi_cols = ['geoid', 'year', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile']
gdf_subset = gdf_annual[svi_cols]
gdf_pivoted = gdf_subset.pivot(index='geoid', columns='year')

In [32]:
num_unique_geoids = len(gdf_pivoted.index)
num_years = len(gdf_pivoted.columns.levels[1])
num_values = len(svi_cols[2:])

numpy_3d_array = gdf_pivoted.values.reshape(num_unique_geoids, num_years, num_values)
svi_path = os.path.join(data_path, 'svi.pkl')
with open(svi_path, "wb") as file:
    pickle.dump(numpy_3d_array, file)

### Add overdose as picke file